### Пункт 5

#### подготовка данных:

In [1]:
import pymorphy2
import json
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from tqdm.auto import tqdm

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
with open("koncheewshop_sv.txt", "r" , encoding="utf-8") as f: # для примера я взял один текст
        some_text = f.read()

In [4]:
df = pd.DataFrame([some_text], columns=["text"])

In [5]:
df["sentences"] = df["text"].apply(lambda x: sent_tokenize(x, "russian"))

In [6]:
df["words"] = df["sentences"].apply(lambda x: [word_tokenize(words) for words in x])

#### извлечение стимулов:

In [7]:
my_verbs = ["знать", "бояться", "подозревать", "думать", "подумать", "чувствовать", "понимать", "понять",
            "надеяться", "забывать", "забыть" , "обнаруживать", "обнаружить", "воображать", "вообразить"]
factives = ["знать", "понимать", "понять", "забывать", "забыть", "обнаруживать", "обнаружить"]
modals = ["возможно", "может", "следует", "мочь", "надо", "нужно", "должен", "желательно"]
conjs = ["кто", "что", "какой", "чей", "сколько"]

Для того чтобы вытащить клаузы (комплементы предположительно матричного глагола) я проверял, есть ли в трех словах после глагола что-нибудь в роли союза или, если не получалось, запятая; если не выходило, то просто брал все, что после глагола, так же поступал в случае с вопросом.

In [17]:
def get_clause(sentence, verb, condition):
    if condition in ["modals", "condition", "negation"]:
        for word in sentence[sentence.index(verb):sentence.index(verb) + 3]:
            if morph.parse(word)[0].normal_form in conjs:
                clause = sentence[sentence.index(word):]
            elif "," in sentence[sentence.index(verb):sentence.index(verb) + 3]:
                comma_relative_index = sentence[sentence.index(verb):sentence.index(verb) + 3].index(",")
                clause = sentence[sentence.index(verb) + comma_relative_index:]
            else:
                clause = sentence[sentence.index(verb) + 1:]
    elif condition == "question":
        clause = sentence[sentence.index(verb) + 1: -1]
    return " ".join(clause)

Для того, чтобы вытащить таргетное предложение, я проверял, есть ли модальный глагол, условный оператор или отрицание в трех словах до предположительно матричного глагола. В случае с вопросом я отбирал примеры со знаком вопроса в конце предложения.

In [18]:
my_annotation = {}
count = 0
for index, sent in tqdm(enumerate(df["words"].iloc[0])):
    for word in sent:
        condition_one = morph.parse(word)[0].normal_form in my_verbs
        condition_modals = (set(sent[sent.index(word)-3:sent.index(word)])
                         & set(modals))
        condition_condition = (set(sent[sent.index(word)-3:sent.index(word)])
                         & set(["если"]))
        condition_negation =(set(sent[sent.index(word)-3:sent.index(word)])
                         & set(["не"]))
        condition_question = sent[-1] == "?"
        if condition_one and (condition_modals or condition_condition
                              or condition_negation or condition_question):
            if condition_modals:
                condition = "modals"
            elif condition_condition:
                condition = "condition"
            elif condition_negation:
                condition = "negation"
            else:
                condition = "question"
            
            clause = get_clause(sent, word, condition)
            is_factive = morph.parse(word)[0].normal_form in factives
            my_annotation[count] = {"target": " ".join(sent), "clause": clause,
                                    "context": df["sentences"].iloc[0][index-2:index], "verb": word,
                                    "embedding": condition, "factive": is_factive,
                                    "tense": morph.parse(word)[0].tag.tense, "genre": "philosophy"}
            count += 1

0it [00:00, ?it/s]

In [19]:
# with open("my_annotation.txt", "w", encoding="utf-8") as f:
#     json.dump(my_annotation, f)

In [20]:
# df_annotation = pd.read_json("my_annotation.txt").T
df_annotation.to_csv("my_annotation.tsv", sep="\t")
df_annotation

,target,clause,context,verb,embedding,factive,tense,genre
0,"Первый и главный аргумент , по отношению к кот...",", ошибочно вообразив , что требуется установит...",[Так как\nАкадемия опубликовала свой отзыв о м...,понял,negation,True,past,philosophy
1,"Но так поймет и должен понять ее всякий , кто ...","ее всякий , кто прочтет тему : это прямо напеч...","[Это и есть, бесспорно, то fundamentum\nphilos...",понять,modals,True,None,philosophy
2,Ибо пока не существует признанная объективной ...,", возможна ли вообще подобная метафизика , а т...",[А\nтут еще присоединяется и то обстоятельство...,знаем,negation,True,pres,philosophy
3,"Ибо , как истинная Академия , она не знает ник...","никаких иных интересов , кроме интересов истин...","[Правда, это не помешало Королевской Норвежско...",знает,negation,True,pres,philosophy
4,я и взялся дать на них ответ . Ибо хотя мне да...,"что я слишком серьезно отношусь к философии , ...","[Опасения за последствия и есть, без\nсомнения...",думал,negation,False,past,philosophy
...,...,...,...,...,...,...,...,...
58,"Помнится , я читал , что один англичанин , зас...",", какой она бросила на него , умирая , и с тех...",[И сострадание это указывает для себя один и т...,забыть,negation,True,None,philosophy
59,В Азии религии дают животным достаточную защит...,о подобного рода обществах .,[Эта \nтонко чувствующая английская нация пере...,думает,negation,False,pres,philosophy
60,"Его члены организуют тайный надзор , чтобы зат...",всюду * * * .,"[К чести англичан надо поэтому \nсказать, что ...",бояться,modals,False,None,philosophy
61,"Требование это и здесь поднимает вопрос , поче...","имеет тот , а не иной вид и каким образом изло...",[Как в конце всякого \nисследования и всякой р...,понятое,question,True,past,philosophy


#### Выводы

Я добавил в аннотацию столбец "Clause", так как это могло бы быть удобно для последующей ручной оценки комитмента. Но реализовать это оказалось непросто, поэтому здесь это скорее как пример, что могло бы получиться. Также, я убрал столбцы "Prompt", так как автоматически его не разметить, и столбцы про субъект матричного предиката, так как определить автоматически субъект без синтаксического парсера очень трудно.

Видно, что правилами отбора стимулов, которые я привел не всегда отбирается то, что нужно (ср., *Невежда , думающий так , никогда раньше так не думал , но в конце концов вынужден был сдаться & # 187 ; 58 ( фр . ) .* или *Пока мы их не могли понять , они не существовали для нашей воли .*); но иногда находится нужное (ср. *Ибо пока не существует признанная объективной и неоспоримая метафизика , т.е.просто метафизика , мы не знаем даже , возможна ли вообще подобная метафизика , а также & # 151 ; какова она будет и может быть .* или *Ибо хотя мне давно уже стало ясно , что я слишком серьезно отношусь к философии , чтобы мог сделаться ее профессором , но я все-таки не думал , что тот же самый порок может повредить мне и в глазах какой-либо академии .*).


Возможное улучшение работа алгоритма, как мне кажется – использование синтаксического парсера. Если через него вводить ограничения на комплемент матричного предиката, то может быть лучше.

Я еще пытался реализовать выделение клауз через spacy отсюда (https://subscription.packtpub.com/book/data/9781838987312/2/ch02lvl1sec13/splitting-sentences-into-clauses),
но на сложных предложениях не лучше работает:

In [83]:
import spacy

In [85]:
nlp = spacy.load("ru_core_news_sm")

In [311]:
sentence = " ".join(df["words"].iloc[0][87])

In [312]:
sentence

'Ибо пока не существует признанная объективной и неоспоримая метафизика , т.е.просто метафизика , мы не знаем даже , возможна ли вообще подобная метафизика , а также & # 151 ; какова она будет и может быть .'

In [111]:
doc = nlp(" ".join(sentence))

In [112]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]

In [113]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [114]:
root_token = find_root_of_sentence(doc)

In [115]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs

In [116]:
other_verbs = find_other_verbs(doc, root_token)

In [117]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)

In [118]:
token_spans = []   
all_verbs = [root_token] + other_verbs
for other_verb in all_verbs:
    (first_token_index, last_token_index) = \
     get_clause_token_span_for_verb(other_verb, 
                                    doc, all_verbs)
    token_spans.append((first_token_index, 
                        last_token_index))

In [119]:
sentence_clauses = []
for token_span in token_spans:
    start = token_span[0]
    end = token_span[1]
    if (start < end):
        clause = doc[start:end]
        sentence_clauses.append(clause)
sentence_clauses = sorted(sentence_clauses, 
                          key=lambda tup: tup[0])

In [121]:
clauses_text = [clause.text for clause in sentence_clauses]
clauses_text

['Ибо пока не существует признанная объективной и неоспоримая метафизика , т.е.просто метафизика',
 'мы не знаем даже , возможна ли вообще подобная метафизика , а также & # 151 ; какова она будет и может быть',
 'и может']